In [1]:
import utils
utils.mount_src()

CONFIG = utils.load_config()
DATA_PATH = CONFIG["images_path"]

In [2]:
from tensorflow.keras import Model
from data_loader import DataLoader

def model_traning_pipeline(model: Model, data_path: str, epochs=10, target_size=(224, 224), callbacks=None):
    data_loader = DataLoader(data_path)
    train_dataset, test_dataset = data_loader.dataset(target_size=target_size)
    
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        callbacks=callbacks
    )
    return {"model": model, "history": history.history}

In [3]:
from tensorflow.keras import layers
import tensorflow as tf

def affine_model():
    inputs = keras.Input(shape=(224, 224, 3))
    x = layers.Rescaling(1./255)(inputs)

    x = layers.Conv2D(filters=32, kernel_size=5, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    x = layers.Conv2D(filters=64, kernel_size=5, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    x = layers.Conv2D(filters=64, kernel_size=5, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    x = layers.Flatten()(x)

    output_year = tf.keras.layers.Dense(1, name="year", activation="sigmoid")(x)
    output_lat = tf.keras.layers.Dense(1, name="lat", activation="sigmoid")(x)
    output_lon = tf.keras.layers.Dense(1, name="lon", activation="sigmoid")(x)

    return tf.keras.Model(inputs=inputs, outputs={"year": output_year, "lat": output_lat, "lon": output_lon})

In [4]:
from tensorflow import keras

affine = affine_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="affine.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

affine.compile(
    optimizer="adam",
    loss={"year": "mse", "lat": "mse", "lon": "mse"},
    metrics={"year": "mae", "lat": "mae", "lon": "mae"}
)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

def efficient_net_model():
    efficient_base = keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(224, 224, 3)
    )

    efficient_base.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))

    x = efficient_base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.3)(x)

    output_year = tf.keras.layers.Dense(1, name="year", activation="sigmoid")(x)
    output_lat = tf.keras.layers.Dense(1, name="lat", activation="sigmoid")(x)
    output_lon = tf.keras.layers.Dense(1, name="lon", activation="sigmoid")(x)

    return tf.keras.Model(inputs=inputs, outputs={"year": output_year, "lat": output_lat, "lon": output_lon})

efficient_net = efficient_net_model()

efficient_net.compile(
    optimizer="adam",
    loss={"year": "mse", "lat": "mse", "lon": "mse"},
    metrics={"year": "mae", "lat": "mae", "lon": "mae"}
)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="affine.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

affine_results = model_traning_pipeline(affine, DATA_PATH)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="effnet.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

effnet_results = model_traning_pipeline(efficient_net, DATA_PATH)

In [ ]:
for k in affine_results["history"]:
    print(k)

In [ ]:
from scaler import Scaler
import tensorflow as tf
import polars as pl
import numpy as np

year = 1962
id = 51871450111
example = DATA_PATH + f"/{year}/{id}.jpg"

def generate_pred_for_image(model: tf.keras.Model, img_path: str, target_size=(224, 224)) -> dict[str, np.ndarray]:
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, target_size) #/ 255.0
    img = tf.expand_dims(img, axis=0)

    pred = model.predict(img)
    return Scaler.descale_preds(pred)

pred = generate_pred_for_image(effnet_results["model"], example)
pred

In [ ]:
df = pl.read_csv(DATA_PATH + "/metadata.csv")
df.filter(pl.col("id") == id)

In [ ]:
from data_loader import DataLoader

data_loader = DataLoader(DATA_PATH)
train_dataset, test_dataset = data_loader.dataset()

for img, label in train_dataset.take(5):
    print(f"Image shape: {img.shape}")
    print(f"Label: {label}")

In [ ]:
import matplotlib.pyplot as plt

for img, label in train_dataset.take(5):
    plt.imshow(img[0])
    plt.title(f"Year: {Scaler.descale_years(label["year"].numpy()[0])}")
    plt.show()